In [0]:
%python
## Import libraries
import pyspark as ps
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *
import pandas as pd
import json

In [0]:
class Error_Handle:
  def __init__(self):
    self.query = "Query About to fill"
    self.sql_df = pd.DataFrame()
    self.data_json = {"Data":[]}    
  
  def logger(message):
    print(message)
    
  def set_query_string(self):
    query = """WITH t(usecase_id, hist_retention) AS (SELECT USECASE_ID,hist_retention from com_us_alyt_ngebox.metadata_usecase)
update com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed  process_table set process_table.Not_Sent_To_Veeva_Reason = (case  when ISNULL(process_table.Not_Sent_To_Veeva_Reason)= true then 'Omni_ID is null' else concat(process_table.Not_Sent_To_Veeva_Reason, '|', 'Omni_ID is null' ) end)  where HCP_OMNI_ID is null and CAST(created_date as DATE) >= date_add(current_date, -(select t.hist_retention from t where t.usecase_id = usecase_id)) and SUGGEST_EXTERNAL_ID_VOD__C in (select p.SUGGEST_EXTERNAL_ID_VOD__C from com_us_alyt_ngebox.ngebox_suggestion_requests_processed p left join com_us_lake.rep_suggestion_vod__c s on p.SUGGEST_EXTERNAL_ID_VOD__C = s.SUGGESTION_EXTERNAL_ID_VOD__C where (s.SUGGESTION_EXTERNAL_ID_VOD__C is NULL) or (s.SUGGESTION_EXTERNAL_ID_VOD__C is not null and s.DISMISSED_VOD__C = 0));
WITH t(usecase_id, hist_retention) AS (SELECT USECASE_ID,hist_retention from com_us_alyt_ngebox.metadata_usecase)
update com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed  process_table set process_table.Not_Sent_To_Veeva_Reason = (case  when ISNULL(process_table.Not_Sent_To_Veeva_Reason)= true then 'NPI is null' else concat(process_table.Not_Sent_To_Veeva_Reason, '|', 'NPI is null' ) end)  where hcp_npi is null and CAST(created_date as DATE) >= date_add(current_date, -(select t.hist_retention from t where t.usecase_id = usecase_id)) and SUGGEST_EXTERNAL_ID_VOD__C in (select p.SUGGEST_EXTERNAL_ID_VOD__C from com_us_alyt_ngebox.ngebox_suggestion_requests_processed p left join com_us_lake.rep_suggestion_vod__c s on p.SUGGEST_EXTERNAL_ID_VOD__C = s.SUGGESTION_EXTERNAL_ID_VOD__C where (s.SUGGESTION_EXTERNAL_ID_VOD__C is NULL) or (s.SUGGESTION_EXTERNAL_ID_VOD__C is not null and s.DISMISSED_VOD__C = 0));
WITH t(usecase_id, hist_retention) AS (SELECT USECASE_ID,hist_retention from com_us_alyt_ngebox.metadata_usecase)
update com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed  process_table set process_table.Not_Sent_To_Veeva_Reason = (case  when ISNULL(process_table.Not_Sent_To_Veeva_Reason)= true then 'ownerid is null' else concat(process_table.Not_Sent_To_Veeva_Reason, '|', 'ownerid is null' ) end)  where ownerid is null and CAST(created_date as DATE) >= date_add(current_date, -(select t.hist_retention from t where t.usecase_id = usecase_id)) and SUGGEST_EXTERNAL_ID_VOD__C in (select p.SUGGEST_EXTERNAL_ID_VOD__C from com_us_alyt_ngebox.ngebox_suggestion_requests_processed p left join com_us_lake.rep_suggestion_vod__c s on p.SUGGEST_EXTERNAL_ID_VOD__C = s.SUGGESTION_EXTERNAL_ID_VOD__C where (s.SUGGESTION_EXTERNAL_ID_VOD__C is NULL) or (s.SUGGESTION_EXTERNAL_ID_VOD__C is not null and s.DISMISSED_VOD__C = 0));
"""
    self.query = query    

  def run_query_and_set_sqldf(self):
    query_list = self.query.split(";")
    self.sql_df = []
    for curr_query in query_list:
      if curr_query.strip() == "":
        continue
      sql_df = sqlContext.sql(curr_query).collect() # type(sql_df) -> dict
      self.sql_df = self.sql_df + sql_df
    
  def format_sqldf_to_json(self):
    sql_output_rowlist = []
    for curr_row in self.sql_df: 
      sql_output_rowlist.append (curr_row.asDict(True))
    sql_df_new = pd.DataFrame(sql_output_rowlist)
    sql_jsonstr = sql_df_new.to_json(orient="records") # type(result) -> str
    sql_jsonparse = json.loads(sql_jsonstr) # type(parsed_json) -> list
    nb_json = {"data":sql_jsonparse} # type(metadata_json) -> dict
    self.nb_json = nb_json
    
  def construct_microservice_response(self):
    resp = {
      "nge_response": {
        "status": 200,
        "body": self.nb_json
      }
    }
    self.response = resp

In [0]:
%python
data_obj = Error_Handle()
data_obj.set_query_string()
data_obj.run_query_and_set_sqldf()
data_obj.format_sqldf_to_json()
data_obj.construct_microservice_response()
dbutils.notebook.exit(
  data_obj.response
)

{'nge_response': {'status': 200, 'body': {'data': [{'num_affected_rows': 2}, {'num_affected_rows': 2}, {'num_affected_rows': 0}]}}}